# 사진에서 정보추출

In [12]:
from PIL import Image
from PIL.ExifTags import TAGS

img_path = 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Homepage/static/images/Upload_Images/20211117_220231.jpg'
image = Image.open(img_path)
info = image._getexif();
image.close()

In [25]:
taglabel = dict()

for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    taglabel[decoded] = value

print(taglabel)
taglabel['DateTime']

{'ImageWidth': 4032, 'ImageLength': 1816, 'ResolutionUnit': 2, 'ExifOffset': 226, 'Make': 'samsung', 'Model': 'SM-G991N', 'Software': 'G991NKSS3AUIN', 'Orientation': 6, 'DateTime': '2021:11:17 22:02:31', 'YCbCrPositioning': 1, 'XResolution': 72.0, 'YResolution': 72.0, 'ExifVersion': b'0220', 'ShutterSpeedValue': 0.030303030303030304, 'ApertureValue': 1.69, 'DateTimeOriginal': '2021:11:17 22:02:31', 'DateTimeDigitized': '2021:11:17 22:02:31', 'ExposureBiasValue': 0.0, 'MaxApertureValue': 1.69, 'MeteringMode': 2, 'ColorSpace': 1, 'Flash': 0, 'FocalLength': 5.4, 'ExifImageWidth': 4032, 'ExifImageHeight': 1816, 'DigitalZoomRatio': 1.0, 'FocalLengthIn35mmFilm': 26, 'SceneCaptureType': 0, 'OffsetTime': '+09:00', 'OffsetTimeOriginal': '+09:00', 'ExposureTime': 0.030303030303030304, 'FNumber': 1.8, 'ImageUniqueID': 'R12LLMF00SM', 'ExposureProgram': 2, 'ISOSpeedRatings': 500, 'ExposureMode': 0, 'WhiteBalance': 0}


'2021:11:17 22:02:31'

In [29]:
date = taglabel['DateTime']
date_list = date.split()
ymd = date_list[0]
date = '날짜 : ' + '. '.join(ymd.split(':'))
date

'날짜 : 2021. 11. 17'